In [14]:
#Weight & Biases
import wandb

#Yaml
from yaml import load, FullLoader

#Embeddings
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *

#Dimensionality reduction
from tsai.imports import *

#Clustering
import hdbscan
import fastai
from fastai.tabular import *
import fastcore
import fastshap
import fastinference
import wwf

In [16]:
check_memory_usage = True
if check_memory_usage:
    import nbs_pipeline.utils.memory as mem
    import torch 
    gpu_device = torch.cuda.current_device()
    mem.gpu_memory_status(gpu_device)
#Get W&B API
api = wandb.Api()

## Buscando una combinación buena
cpu_flag = False

if cpu_flag:
    n_neighbors = 15
    min_dist = 0.1
else: 
    n_neighbors = 15 #200
    min_dist = 0.1 #0.0001
config = AttrDict(
    use_wandb = False, # Whether to use or not wandb for experiment tracking
    wandb_group = None, # Whether to group this run in a wandb group
    wandb_entity = os.environ['WANDB_ENTITY'], # The entity to use for wandb,
    wandb_project = os.environ['WANDB_PROJECT'], # The project to use for wandb,
    dr_artifact_name = None, # * Set to None for using the default one (encoder validation set)
    enc_artifact = 'mi-santamaria/deepvats/mvp-SWV:latest', # Name:version of the encoder artifact
    n_neighbors = n_neighbors, #15, #UMAP
    min_dist = min_dist, #0.1, #UMAP,
    random_state = int(1234), # UMAP
    metric = 'euclidean',
    cpu = cpu_flag
)
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="to_delete"
runname = name
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"

run_dr = wandb.init(
    entity=config.wandb_entity,
    project=config.wandb_project if config.use_wandb else 'work-nbs', 
    group=config.wandb_group,
    allow_val_change=True, 
    job_type='dimensionality_reduction', 
    mode='online' if config.use_wandb else 'disabled',
    anonymous = 'never' if config.use_wandb else 'must',
    config=config,
    resume = 'allow',
    name = runname
    #resume=False
)
config_dr = wandb.config # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config_dr.use_wandb else api.artifact
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

enc_logger = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
if enc_logger.config['valid_artifact'] is not None:
    enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
    print("enc_artifact_valid:", enc_artifact_valid.name)
    print("enc_artifact_train: ", enc_artifact_train.name)


if config_dr.dr_artifact_name is not None:
    dr_artifact = artifacts_gettr(config_dr.dr_artifact_name)
else:
    dr_artifact = enc_artifact_train
df = dr_artifact.to_df()

GPU | Used mem: 5937
GPU | Used mem: 24576
GPU | Memory Usage: [████----------------] 24%


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [18]:
df.head()

,0
2019-08-01 00:00:03,0.0
2019-08-01 00:00:23,0.0
2019-08-01 00:00:43,0.0
2019-08-01 00:01:03,0.0
2019-08-01 00:01:23,0.0


In [19]:
df.iloc[10]

0    0.0
Name: 2019-08-01 00:03:23, dtype: float64

In [20]:
enc_learner

In [22]:
exp = fastinference.ShapInterpretation(learn, df.iloc[:100])

AttributeError: module 'fastinference' has no attribute 'ShapInterpretation'